In [22]:
import numpy as np
from numpy import *
import sympy
from sympy import *

In [23]:
import auxiliars
from auxiliars import *

In [24]:
import auxiliars2
from auxiliars2 import *

In [55]:
class NumericalSemigroup:
    '''
    This class contains all methods and properties needs
    for working with a Numerical Semigroup.
    
    In order to use it you have to introduce the generators. For example S = NumericalSemigroup([3,4,5]).
    
    Functions:
        * FrobeniusNumber() returns the frobenius number of S.
        * Factorizations(x) returns the factorizations of x in S.
        * Belongs(x) returns True or False if x is or is not in S.
        * ComputeNS() returns a bound for the periodicity of Delta(S)
    '''
    
    def __init__(self,generators):
        self.generators = generators
        self.smgS()
        self.multiplicity = generators[0]
        self.eDimension = len(generators)
        
    # Frobenius Number
    fNumber = 0
    # Bound for Delta(S) periodic
    NS = 0
    
    # This function compute a minial system of generators.
    def smgS(self):
        smgS=array([],dtype=np.int)
        self.generators.sort()
        #print(sgS1)
        sgS=np.unique(self.generators)
        #print(sgS)
        if len(sgS)==1:
            return sgS
        smgS=np.append(smgS,sgS[0])
        #print(sgS,smgS)
        for i in range(1,len(sgS)):
            if sgS[i] % smgS[0] != 0:
                smgS=np.append(smgS,sgS[i])
                break
        #print(i,sgS,smgS)
        for j in range(i+1,len(sgS)):
            if len(FSolve(smgS,sgS[j]))==0:
                smgS=np.append(smgS,sgS[j])
        self.generators = list(smgS)
    
    # This function gives us the Frobenius Number of a semigroup
    def FrobeniusNumber(self):
        if self.fNumber != 0:
            return fNumber
        else:
            self.fNumber = FrobeniusNumber(self.generators,self.eDimension)
        return self.fNumber
    
    def Factorizations(self,x):
        posAModificar=0
        sumando=True
        dim = self.eDimension
        ceros=np.array([0 for i in range(dim)],dtype=np.int)
        xaux=x
        tuplaActual=array([0 for i in range(dim)],dtype=np.int)   
        if x==0:
            return tuplaActual
        soluciones=[]
        while( not( np.all(tuplaActual==ceros) and not(sumando) ) ):
            if sumando:
                if xaux>=self.generators[0]:
                    tuplaActual[posAModificar]=tuplaActual[posAModificar]+1
                    xaux=xaux-self.generators[posAModificar]
                    if xaux==0:
                        soluciones.append(array(tuplaActual))
                        sumando=False
                        continue
                if xaux!=0 and xaux<self.generators[0]:
                    sumando=False
                    continue
            else:
                if ( posAModificar == dim-1 ) and ( tuplaActual[posAModificar] > 0 ):
                    xaux=tuplaActual[posAModificar]*self.generators[posAModificar]+xaux
                    tuplaActual[posAModificar]=0
                    posAModificar=posAModificar-1
                    continue
                if ( posAModificar < dim-1 ) and ( tuplaActual[posAModificar] > 0 ):
                    xaux=xaux+self.generators[posAModificar]
                    tuplaActual[posAModificar]=tuplaActual[posAModificar]-1
                    posAModificar=posAModificar+1
                    sumando=True
                    continue
                if ( posAModificar < dim-1 ) and ( tuplaActual[posAModificar] == 0 ):
                    posAModificar=np.max( [i for i in range(dim) if tuplaActual[i]!=0] )
                    xaux=self.generators[posAModificar]+xaux
                    tuplaActual[posAModificar]=tuplaActual[posAModificar]-1
                    sumando=True
                    posAModificar=posAModificar+1
                    continue
        return [list(x) for x in soluciones]
    
    # This function check if a number is in the semigroup
    def Belongs(self,x):
        if x==0:
            return True
        if 0<x and x<self.multiplicity:
            return False
        if x in self.generators:
            return True
        if self.fNumber != 0 and x>self.fNumber:
            return True
        expression = self.Factorizations(x)
        if len(expression)>0:
            return True
        else:
            return False
        
    def ComputeNS(self):
        if self.NS != 0:
            return NS
        # Calculamos la dimension
        a = self.generators
        dimension = self.eDimension
        # Creamos los simbolos ei
        e = symbols('e0:%d'%dimension)
        # Creamos el simbolo s
        s = symbols('s')
        # Calculamos d
        m = equationsToGeneratorsHomogeneusCase(Matrix([self.generators]))
        m1 = sympyMatrix2numpyArray(m)
        l = numpy.sum(m1,axis=1) # Esto es lo que fallaba porque l es de la forma [a b c] y deberia ser [a, b, c] con las comas, asi que lo transformo aqui.
        lista = []
        for i in range(len(l)):
            lista.append(l[i])
        d = gcdL(lista)
        # Calculo un vector con los mcd
        mcd = []
        for i in range(dimension-2):
            mcd.append(gcdL([a[i+1]-a[-1],-a[0]+a[-1],a[0]-a[i+1]]))
        # Calculo h
        h=d/(a[-1]-a[0])*(a[-1]*e[0]-a[0]*e[-1])
        # Calculo P2
        P2=s*(a[1]-a[-1])/(a[1]*(a[0]-a[-1]))*e[0]+s*(a[0]-a[1])/(a[1]*(a[0]-a[-1]))*e[-1]
        # Calculo Pp-1
        Ppm1=s*(a[-2]-a[-1])/(a[-2]*(a[0]-a[-1]))*e[0]+s*(a[0]-a[-2])/(a[-2]*(a[0]-a[-1]))*e[-1]
        # Calculo un vector con los qi
        Q = []
        for i in range(dimension-2):
            Q.append(1/mcd[i]*((a[i+1]-a[-1])*e[0]+(a[-1]-a[0])*e[i+1]+(a[0]-a[i+1])*e[-1]))
        # Calculo la suma de los Qi
        sumaQi = 0
        for i in range(dimension-2):
            sumaQi += Q[i]
        # Calculamos la última coordenada P2+h+∑qi
        primeraEcuacion = P2+h+sumaQi
        for i in range(dimension-1):
            primeraEcuacion = primeraEcuacion.subs(e[i],0)
        primeraEcuacion = primeraEcuacion.subs(e[dimension-1],1)
        sol1 = sympy.solve(primeraEcuacion,s)
        # Calculamos la primera coordenada de P(p−1)−h+∑qi
        ultimaEcuacion = Ppm1-h+sumaQi
        for i in range(1,dimension):
            ultimaEcuacion = ultimaEcuacion.subs(e[i],0)
        ultimaEcuacion = ultimaEcuacion.subs(e[0],1)
        sol2 = sympy.solve(ultimaEcuacion,s)
        self.NS = int(ceil(int(max(sol1,sol2)[0])))
        return self.NS
    def ComputeN0(self):
        try:
            return self.N0
        except (AttributeError, NameError):
            Ns=self.ComputeNS()
            a1=self.generators[0]
            a2=self.generators[1]
            apM1=self.generators[-2]
            ap=self.generators[-1]
            C1=(ap-apM1)*Ns/apM1
            C2=(ap-a2)*Ns/a2
            C3=(-ap/a1+ap/a2-ap/apM1+1)*Ns
            C4=(a1/apM1-a1/ap-a1/a2+1)*Ns
            self.lambda1=max(C1,C4)
            self.lambda2=max(C2,C3)
            self.N0=max([Ns/a1,(ap-a1+self.lambda1+self.lambda2)/(ap-a1)])
            return self.N0

In [56]:
ns = NumericalSemigroup([5,6,7,8])

In [57]:
ns.FrobeniusNumber()

9

In [58]:
ns.ComputeN0()

24.375

In [62]:
ns.NS

119

In [7]:
ns.Factorizations(30)

[[6, 0, 0, 0],
 [3, 0, 1, 1],
 [2, 2, 0, 1],
 [2, 1, 2, 0],
 [1, 3, 1, 0],
 [0, 5, 0, 0],
 [0, 1, 0, 3],
 [0, 0, 2, 2]]

In [ ]:
ns.Belongs(9)

In [ ]:
ns.ComputeNS()